In [ ]:
#建立攝影機
import requests

media_server_ip = input("請輸入Ant Media Server IP (e.g. 61.222.163.86):")
media_server_port = int(input("請輸入 Port (預設 5080):") or 5080)
username = input("請輸入username (e.g. admin)")
password = input("請輸入password(e.g. ez008313)")
zone = input("請輸入機台區域 (e.g. A/B/C/D):").strip().upper()
subnet = int(input("請輸入網段(e.g. 11/12/13/14):").strip())
start_id = int(input("請輸入起始 ID (包含):"))
end_id = int(input("請輸入結束 ID (包含):"))

#ip變數規則
def ip_location(subnet: str, id: int) -> int:
    if subnet == "11":
        return id
    elif subnet == "12":
        return id - 200
    elif subnet in ("14", "15"):
        return id - 300
    elif subnet == "13":
        if id > 400:
            return id - 400
        elif id > 300:
            return id - 300
        else:
            return id
    else:
        raise ValueError(f"Unknown subnet: {subnet}")

results = []
fail_count = 0

for id in range(start_id, end_id + 1):
    suffix = ip_location(subnet, id)
    ip_addr = f"192.168.{subnet}.{suffix}"
    name = f"{zone}{id if id > 100 else f'0{id}'}"

    url = f"http://{media_server_ip}:{media_server_port}/WebRTCAppEE/rest/v2/broadcasts/create?autoStart=true"
    data = {
        "hlsViewerCount": 0,
        "dashViewerCount": 0,
        "webRTCViewerCount": 0,
        "rtmpViewerCount": 0,
        "mp4Enabled": 0,
        "playlistLoopEnabled": True,
        "autoStartStopEnabled": False,
        "plannedStartDate": 0,
        "playListItemList": [],
        "ipAddr": ip_addr,
        "name": name,
        "username": username,
        "password": password,
        "type": "ipCamera",
    }

    response = requests.post(url, json=data, timeout= 10)

    if response.status_code == 200:
        results.append({'id': id, 'name': name})
    else:
        fail_count += 1
        print(f"❌ {name} 建立失敗:HTTP {response.status_code} → {response.text}")

success_count = len(results)
total_count = end_id - start_id +1
print(f"✅ 建立完成: 共{total_count}，成功 {success_count} 筆，失敗{fail_count}筆。")
